In [2]:
from datasets import load_dataset
import pickle
from collections import Counter

In [3]:
conll_dataset = load_dataset("conll2003")

Reusing dataset conll2003 (/opt/tmp/huggingface/datasets/conll2003/conll2003/1.0.0/26b70ce2b0f32cb35a27151dbfa2dbe88c82bcdaf8f29433bcdc612a9b314e83)


In [30]:
subclass2labels = pickle.load(open('../subclass2labels.pickle', 'rb'))
label2subclasses = pickle.load(open('../label2subclasses.pickle', 'rb'))
subclass2class = pickle.load(open('../subclass2class.pickle', 'rb'))

In [126]:
subclass2labels['dynasty']

{'albero',
 'askia',
 'bonacossi',
 'borjigin',
 'frataraka',
 'jabach',
 'nakharar',
 'papareschi',
 'q61055282'}

In [151]:
subclass2class['steve']

KeyError: 'steve'

In [129]:
list(label2subclasses.keys())[:10]

['askia',
 'jabach',
 'papareschi',
 'bonacossi',
 'frataraka',
 'q61055282',
 'borjigin',
 'nakharar',
 'albero',
 'q10308379']

In [157]:
label2subclasses['chomsky']

['musical ensemble', 'family name']

In [136]:
label2classes = {l:list(set([c if c not in subclass2class else subclass2class[c] for c in cs])) for l, cs in label2subclasses.items()}

In [137]:
label2classes['ismail']

['given name']

In [138]:
pickle.dump(label2classes, open('word2categories.pickle', 'wb'))

In [71]:
for i, (label, l) in enumerate(sorted(subclass2labels.items(), key= lambda l: -len(l[1]))):
    print(f"{i+1:3}. {label} : {len(l)}")
    if i > 50: break

  1. human settlement : 462774
  2. family name : 313954
  3. hill : 99328
  4. mountain : 96785
  5. business : 51620
  6. valley : 38020
  7. organization : 34978
  8. locality : 32642
  9. male given name : 28307
 10. geolocation : 24255
 11. town in china : 18607
 12. library : 17975
 13. female given name : 17358
 14. enterprise : 16058
 15. given name : 14872
 16. radio station : 14862
 17. georegion : 14059
 18. town : 11143
 19. city : 10045
 20. rural council of ukraine : 9945
 21. park : 8657
 22. magazine : 7878
 23. village : 7677
 24. newspaper : 7665
 25. summit : 7511
 26. software : 7156
 27. settlement in galicia, spain : 6648
 28. monastery : 5773
 29. family : 5384
 30. sports club : 5313
 31. city of the united states : 5062
 32. television station : 4890
 33. product : 4747
 34. buddhist temple : 4650
 35. voluntary association : 4472
 36. publisher : 4436
 37. census-designated place : 4417
 38. community health center : 4164
 39. name : 3893
 40. protected area :

In [110]:
tag2subclasses = {'ORG': [], 'PER': [], 'LOC': [], 'MISC':[]}
subclass2tags  = {}
not_found = list()
counter = 0
capitalized_counter = 0
all_list = []

for split, data in conll_dataset.items():
    for doc in data:
        for word, ner in zip(doc['words'], doc['ner']):
            tag  = ner.split('-')[-1]
            if word[0] == word[0].upper() and  tag != 'O':
                capitalized_counter += 1
            
            word = word.lower()

            if tag != 'O':
                counter += 1
                if word not in label2subclasses:
                    not_found.append(word)
                    continue
                
                subclasses = label2subclasses[word]
                for subclass in subclasses:
                    subclass = subclass if subclass not in subclass2class else subclass2class[subclass]
                    if subclass not in subclass2tags:
                        subclass2tags[subclass] = []
                    subclass2tags[subclass].append(tag)
                    tag2subclasses[tag].append(subclass)
                    all_list.append(subclass)

mc = 20
tag_counter = {tag:Counter(l).most_common(mc*5) for tag, l in tag2subclasses.items()}
subclass_counter = {subclass:Counter(l).most_common(mc) for subclass, l in subclass2tags.items()}

In [112]:
for i, (label, l) in enumerate(Counter(all_list).most_common()):
    print(f"{i+1:3}. {label} : {l}")

  1. georegion : 39974
  2. family name : 28747
  3. organization : 21161
  4. given name : 18690
  5. city : 10648
  6. country : 9746
  7. town : 8778
  8. geolocation : 6977
  9. product : 3899
 10. name : 3536
 11. brand : 1847
 12. demonym : 1012
 13. capital : 599
 14. artist : 25


In [113]:
capitalized_counter

50094

In [114]:
counter

50758

In [115]:
len(not_found)

11865

In [116]:
'city of the united states'


'city of the united states'

In [117]:
tag_counter

{'ORG': [('georegion', 12978),
  ('organization', 7148),
  ('family name', 5842),
  ('city', 4614),
  ('given name', 3028),
  ('town', 2762),
  ('geolocation', 1956),
  ('product', 1028),
  ('brand', 821),
  ('name', 616),
  ('country', 311),
  ('capital', 250),
  ('demonym', 58),
  ('artist', 12)],
 'PER': [('family name', 13628),
  ('given name', 11260),
  ('georegion', 10309),
  ('organization', 3464),
  ('geolocation', 2314),
  ('town', 2296),
  ('name', 1971),
  ('product', 807),
  ('city', 719),
  ('brand', 507),
  ('country', 42),
  ('capital', 37),
  ('artist', 8),
  ('demonym', 1)],
 'LOC': [('georegion', 15191),
  ('country', 9351),
  ('organization', 8867),
  ('family name', 6567),
  ('city', 5233),
  ('given name', 3478),
  ('town', 3468),
  ('geolocation', 2432),
  ('product', 995),
  ('name', 585),
  ('brand', 345),
  ('capital', 301),
  ('artist', 5),
  ('demonym', 4)],
 'MISC': [('family name', 2710),
  ('organization', 1682),
  ('georegion', 1496),
  ('product', 1069),

In [118]:
subclass2class['male given name']

'given name'

In [119]:
all_subclasses_counter = {tag:Counter(l).most_common(mc*5) for tag, l in tag2subclasses.items()}


In [120]:
tag_counter

{'ORG': [('georegion', 12978),
  ('organization', 7148),
  ('family name', 5842),
  ('city', 4614),
  ('given name', 3028),
  ('town', 2762),
  ('geolocation', 1956),
  ('product', 1028),
  ('brand', 821),
  ('name', 616),
  ('country', 311),
  ('capital', 250),
  ('demonym', 58),
  ('artist', 12)],
 'PER': [('family name', 13628),
  ('given name', 11260),
  ('georegion', 10309),
  ('organization', 3464),
  ('geolocation', 2314),
  ('town', 2296),
  ('name', 1971),
  ('product', 807),
  ('city', 719),
  ('brand', 507),
  ('country', 42),
  ('capital', 37),
  ('artist', 8),
  ('demonym', 1)],
 'LOC': [('georegion', 15191),
  ('country', 9351),
  ('organization', 8867),
  ('family name', 6567),
  ('city', 5233),
  ('given name', 3478),
  ('town', 3468),
  ('geolocation', 2432),
  ('product', 995),
  ('name', 585),
  ('brand', 345),
  ('capital', 301),
  ('artist', 5),
  ('demonym', 4)],
 'MISC': [('family name', 2710),
  ('organization', 1682),
  ('georegion', 1496),
  ('product', 1069),

In [121]:
for i, (label, l) in enumerate(sorted(subclass2labels.items(), key= lambda l: -len(l[1]))):
    print(f"{i+1:3}. {label} : {len(l)}")
    if i == 100: break

  1. human settlement : 462774
  2. family name : 313954
  3. hill : 99328
  4. mountain : 96785
  5. business : 51620
  6. valley : 38020
  7. organization : 34978
  8. locality : 32642
  9. male given name : 28307
 10. geolocation : 24255
 11. town in china : 18607
 12. library : 17975
 13. female given name : 17358
 14. enterprise : 16058
 15. given name : 14872
 16. radio station : 14862
 17. georegion : 14059
 18. town : 11143
 19. city : 10045
 20. rural council of ukraine : 9945
 21. park : 8657
 22. magazine : 7878
 23. village : 7677
 24. newspaper : 7665
 25. summit : 7511
 26. software : 7156
 27. settlement in galicia, spain : 6648
 28. monastery : 5773
 29. family : 5384
 30. sports club : 5313
 31. city of the united states : 5062
 32. television station : 4890
 33. product : 4747
 34. buddhist temple : 4650
 35. voluntary association : 4472
 36. publisher : 4436
 37. census-designated place : 4417
 38. community health center : 4164
 39. name : 3893
 40. protected area :

In [122]:
for i, (label, l) in enumerate(sorted(subclass_counter.items(), key= lambda l: -sum([x[1] for x in l[1]]))):
    print(f"{i+1:3}. {label} : {sum([x[1] for x in l])} {', '.join(map(str, l))}")

  1. georegion : 39974 ('LOC', 15191), ('ORG', 12978), ('PER', 10309), ('MISC', 1496)
  2. family name : 28747 ('PER', 13628), ('LOC', 6567), ('ORG', 5842), ('MISC', 2710)
  3. organization : 21161 ('LOC', 8867), ('ORG', 7148), ('PER', 3464), ('MISC', 1682)
  4. given name : 18690 ('PER', 11260), ('LOC', 3478), ('ORG', 3028), ('MISC', 924)
  5. city : 10648 ('LOC', 5233), ('ORG', 4614), ('PER', 719), ('MISC', 82)
  6. country : 9746 ('LOC', 9351), ('ORG', 311), ('PER', 42), ('MISC', 42)
  7. town : 8778 ('LOC', 3468), ('ORG', 2762), ('PER', 2296), ('MISC', 252)
  8. geolocation : 6977 ('LOC', 2432), ('PER', 2314), ('ORG', 1956), ('MISC', 275)
  9. product : 3899 ('MISC', 1069), ('ORG', 1028), ('LOC', 995), ('PER', 807)
 10. name : 3536 ('PER', 1971), ('ORG', 616), ('LOC', 585), ('MISC', 364)
 11. brand : 1847 ('ORG', 821), ('PER', 507), ('LOC', 345), ('MISC', 174)
 12. demonym : 1012 ('MISC', 949), ('ORG', 58), ('LOC', 4), ('PER', 1)
 13. capital : 599 ('LOC', 301), ('ORG', 250), ('PER

In [50]:
len(not_found)

11865

# Counting O's

In [99]:
O2subclasses = {'O': []}
subclass2O  = {}
O_capitalized_counter = 0
O_capitalized_and_tagged_counter = 0
O_not_found = []
O_counter = 0

for split, data in conll_dataset.items():
    for doc in data:
        for word, ner in zip(doc['words'], doc['ner']):
            tag  = ner.split('-')[-1]
            if word[0] == word[0].upper(): 
                O_capitalized_counter += 1
            if tag == 'O':
                O_counter += 1
                if word not in label2subclasses:
                    O_not_found.append(word.lower())
                    continue
                subclasses = label2subclasses[word.lower()]
                if word[0] == word[0].upper(): 
                    O_capitalized_and_tagged_counter += 1

                    for subclass in subclasses:
                        subclass = subclass if subclass not in subclass2class else subclass2class[subclass]
                        if subclass not in subclass2O:
                            subclass2O[subclass] = []
                        subclass2O[subclass].append(tag)
                        O2subclasses[tag].append(subclass)

mc = 20
O_tag_counter = {tag:Counter(l).most_common(mc*5) for tag, l in O2subclasses.items()}
O_subclass_counter = {subclass:Counter(l).most_common(mc) for subclass, l in subclass2O.items()}

In [100]:
O_capitalized_counter

137019

In [101]:
O_capitalized_and_tagged_counter

3998

In [102]:
O_tag_counter

{'O': [('organization', 5749),
  ('brand', 1898),
  ('name', 1270),
  ('product', 784),
  ('geolocation', 37)]}

In [103]:
for i, (label, l) in enumerate(sorted(O_subclass_counter.items(), key= lambda l: -sum([x[1] for x in l[1]]))):
    print(f"{i+1:3}. {label} : {sum([x[1] for x in l])} {', '.join(map(str, l))}")

  1. organization : 5749 ('O', 5749)
  2. brand : 1898 ('O', 1898)
  3. name : 1270 ('O', 1270)
  4. product : 784 ('O', 784)
  5. geolocation : 37 ('O', 37)


In [104]:
[l for l in subclass2class if subclass2class[l] == 'organization']

['oligarchy',
 'nonprofit organization',
 'order',
 'amusement park',
 'certificate authority',
 'public-private partnership',
 'secret society',
 'government agency',
 'festival',
 'think tank',
 'triad',
 'institution',
 'advocacy group',
 'regional internet registry',
 'international organization',
 'chapter',
 'aid agency',
 'organizational unit',
 'company',
 'financial institution',
 'retinue',
 'alliance',
 'contract research organization',
 'organ',
 'library association',
 'club',
 'umbrella organization',
 'delegation',
 'credit bureau',
 'parliamentary opposition',
 'political campaign',
 'summer camp',
 'oberpostdirektion',
 'consortium',
 'domain',
 'resistance movement',
 'harem',
 'unification church',
 'team',
 'nation',
 'ss-hauptamt',
 'ward',
 'artel',
 'exchange',
 'cathedral works',
 'socrates programme',
 'labor organisation',
 'key distribution center',
 'artist-in-residence',
 'emergency service',
 'guildhalls of venice',
 'fraternal organization',
 'nahdlatul u